## ■ 질병잎파리 구별하는 홈페이지 만들기 

### 10기 포트폴리오 게시글 82번 참고

https://drive.google.com/file/d/1H2XjOzTPx1BpTWmoSHqZYd7naMLBuvb8/view?usp=sharing

## ■ 다른 사진으로 이미지 분류하는 인공신경망 홈페이지 만들기

### ※ 이력서에 쓸때 : 자신만의 이미지 인공신경망 분류 홈페이지 구현 (이상적!!)


1. 이미지를 로드하는 파이썬 함수 ---> loader2.py
2. 학습하는 keras 신경망 ---> 한눈에 이해되는 신경망 코드(게시글 82번)
    - leaf.h5를 yys26 폴더에 가져온다.
3. 화면을 구현하는 ui R 코드 ---> 코드 있음!

## ■ 질병 잎파리 모델 학습시키는 코드 (파이썬)

__" 학습된 매개변수들의 값을 저장하기 위한 용도 "__

In [ ]:
# R로 실행시킬때는 한글부분은 다 삭제시키고 돌려볼 것
from keras.datasets import cifar10
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import loader2 # loader.py를 파이썬 워킹 디렉토리에 가져다 두어야됨 (딥러닝 수업시간에 배움.)

batch_size = 28
num_classes = 2 # 분류할 종류숫자에 따라 조정하면됨.
epochs = 100

# 사진의 크기가 224 x 224 면 메모리가 터지므로( out of memory) 32 x 32로 resize해서 학습시킨다.
# csv 파일도 있어야됨. (label.csv를 만들어주어야됨. 이것도 딥러닝 수업시간에 배웠음.)
# test의 경우 500개 1(정상) 0(비정상)

# 자신에게 맞는 경로로 설정하기
test_image = 'd:\\leaf\\test_resize'
test_label = 'd:\\leaf\\test_label.csv'
train_image = 'd:\\leaf\\train_resize'
train_label = 'd:\\leaf\\train_label.csv'


x_train = loader2.image_load(train_image)
y_train = loader2.label_load(train_label)
x_test = loader2.image_load(test_image)
y_test = loader2.label_load(test_label)      


#(x_train, y_train), (x_test, y_test) = cifar10.load_data() # cifar10 읽어오는건 막음
# One hot Encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 현재 6층 신경망임

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 정규화 작업
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=batch_size, verbose=2)
 
scores = model.evaluate(x_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

# GPU있는 컴퓨터에서 원하는 위치로 설정!
# resize 32x32 19000개 90분 ---> 1000개 약 4분30초
save_model(model, "c:\\data9\\leaf.h5")

## ■ 3. 화면을 구현하는 R 코드 ---> 코드 있음! ( Local 전용 )

In [ ]:
# 이 위치만 조절해주면 clear
# R로 실행시킬때는 한글부분은 다 삭제시키고 돌려볼 것

setwd("D:\\yys26")

packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
model<<-load_model_hdf5("leaf.h5")

synsets <<- readLines("synset.txt")

server <- shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'leaf.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'leaf.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'leaf.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'leaf.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    img <- image_load(src, target_size = c(32,32)) # 파이썬으로 훈련시킨 이미지 사이즈에 맞게 숫자를 바꾸어 주어야 됨.
    img
    x <- image_to_array(img)
    # ensure we have a 4d tensor with single element in the batch dimension,
    x <- array_reshape(x, c(1, dim(x)))
    
    # normalize
    x[,,,1] <- x[,,,1] /255.0
    x[,,,2] <- x[,,,2] /255.0
    x[,,,3] <- x[,,,3] /255.0
    
    # predcit
    preds <- model %>% predict(x)
    
    # output result as string
    
    order(preds[1,], decreasing = TRUE)
    max.idx <- order(preds[1,], decreasing = TRUE)[1]
    max.idx
    result <- synsets[max.idx]
    res_str <- ""
    tmp <- strsplit(result[1], " ")[[1]]
    res_str <- paste0(res_str, tmp[2])
    res_str
  })
})




require(imager)
require(shiny)
require(jpeg)
require(png)

ui <- shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Image classificaiton using DenseNet',
                  windowTitle = 'Image Classification using DenseNet'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - Image Classification")),
               h3(titlePanel("Image Classification"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))

shinyApp(ui = ui, server = server)

### 만약에 tensorflow등 모듈이 없을시


시작 -> 아나콘다 -> anaconda prompt(R-MINI~1) --> tesorflow==1.14 설치

## ■ Ui.R과 Server.R 파일 만들어서 구별하기

## UI.R

In [ ]:
shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Image classificaiton using DenseNet',
                  windowTitle = 'Image Classification using DenseNet'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - Image Classification")),
               h3(titlePanel("Image Classification"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))

## Server.R

In [ ]:


packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
model<<-load_model_hdf5("leaf.h5")

synsets <<- readLines("synset.txt")

shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'leaf.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'leaf.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'leaf.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'leaf.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    img <- image_load(src, target_size = c(32,32)) 
    img
    x <- image_to_array(img)
    # ensure we have a 4d tensor with single element in the batch dimension,
    x <- array_reshape(x, c(1, dim(x)))
    
    # normalize
    x[,,,1] <- x[,,,1] /255.0
    x[,,,2] <- x[,,,2] /255.0
    x[,,,3] <- x[,,,3] /255.0
    
    # predcit
    preds <- model %>% predict(x)
    
    # output result as string
    
    order(preds[1,], decreasing = TRUE)
    max.idx <- order(preds[1,], decreasing = TRUE)[1]
    max.idx
    result <- synsets[max.idx]
    res_str <- ""
    tmp <- strsplit(result[1], " ")[[1]]
    res_str <- paste0(res_str, tmp[2])
    res_str
  })
})




require(imager)
require(shiny)
require(jpeg)
require(png)

## ■ 홈페이지 빌드업하기

In [ ]:
#install.packages('rsconnect')



library(rsconnect)


# personal token
rsconnect::setAccountInfo(name='lullaby0419', 
                          token='76A6ABEE4913A7103F13E9CB0DBD604D', 
                          secret='보안')





rsconnect::deployApp('d:\\yys26',appName = "myapp26")